# Финальный проект - A/B тестирование

## Введение

**Описание проекта**

Наша задача — провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов. Нам надо:

1. Оценить корректность проведения теста.
2. Проанализировать результаты теста.

Чтобы оценить корректность проведения теста, проверим:

1. Пересечение тестовой аудитории с конкурирующим тестом.
2. Совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

**Техническое задание**

1. Название теста: `recommender_system_test`.
2. Группы: А — контрольная, B — новая платёжная воронка.
3. Дата запуска: `2020-12-07`.
4. Дата остановки набора новых пользователей: `2020-12-21`.
5. Дата остановки: `2021-01-04`.
6. Фудитория: 15% новых пользователей из региона EU.
7. Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы.
8. Ожидаемое количество участников теста: 6000.
9. Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%: конверсии в просмотр карточек товаров — событие product_page, просмотры корзины — product_cart, покупки — purchase. 

**Описание данных**

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:
1. `name` — название маркетингового события.
2. `regions` — регионы, в которых будет проводиться рекламная кампания.
3. `start_dt` — дата начала кампании.
4. `finish_dt` — дата завершения кампании.
   
`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `first_date` — дата регистрации.
3. `region` — регион пользователя.
4. `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `event_dt` — дата и время покупки.
3. `event_name` — тип события.
4. `details` — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

`final_ab_participants.csv` — таблица участников тестов.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `ab_test` — название теста.
3. `group` — группа пользователя.

**План работы**

1. Исследование данных: преобразование типов, поиск пропущенных значений и дубликатов.
2. Оценить корректность проведения теста. Проверить:
   1. Соответствие данных требованиям технического задания. Корректность всех пунктов технического задания.
   2. Время проведения теста: оно не должно совпадать с маркетинговыми и другими активностями.
   3. Аудиторию теста. Удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверить равномерность распределения по тестовым группам и правильность их формирования.
3. Провести исследовательский анализ данных:
   1. Количество событий на пользователя одинаково распределены в выборках?
   2. Как число событий в выборках распределено по дням?
   3. Как меняется конверсия в воронке в выборках на разных этапах?
   4. Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
4. Оценить результаты A/B-тестирования:
   1. Что можно сказать про результаты A/В-тестирования?
   2. Проверить статистическую разницу долей z-критерием.
5. Сделать общее заключение о корректности проведения теста.

In [57]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st
from IPython.display import display

# Save raw datasets in case we need them
try:
    raw_new_users = pd.read_csv('final_ab_new_users.csv')
    raw_test_users = pd.read_csv('final_ab_participants.csv')
    raw_events = pd.read_csv('final_ab_events.csv')
    raw_marketing = pd.read_csv('ab_project_marketing_events.csv')
except:
    raw_new_users = pd.read_csv('/datasets/final_ab_new_users.csv')
    raw_test_users = pd.read_csv('/datasets/final_ab_participants.csv')
    raw_events = pd.read_csv('/datasets/final_ab_events.csv')
    raw_marketing = pd.read_csv('/datasets/ab_project_marketing_events.csv')

# Constants, which we will need later
FIG_WIDTH = 8
FIG_HEIGHT = 5


## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Названия колонок.
2. Неподходящие под тип данных значения в колонках.
3. Артефакты в данных и повторяющиеся строки.
4. Пропущенные значения.
5. Дополнительные колонки и типы данных.

Пойдем по порядку.

In [58]:
# Check columns
for dataset in [raw_new_users, raw_test_users, raw_events, raw_marketing]:
    print('-' * 50)
    dataset.info()


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column   

Название колонок уже заданы нормально. У нас есть много пропущенных занчений в датасете `raw_events`, но это может быть связано с тем, как были собраны данные. Наконец, надо будет задать типы данных для колонок с датами.

Посмотрим, что за данные у нас внутри этих таблиц:

In [59]:
(raw_new_users
 .fillna('NA')
 .pivot_table(index='region', columns='device', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .assign(sort_key = lambda df: df.region == 'All')
 .sort_values(['sort_key', 'All'], ascending=[True, False])
 .drop('sort_key', axis=1)
)


index,region,Android,Mac,PC,iPhone,All
2,EU,20629,4575,11693,9373,46270
3,N.America,4077,883,2327,1868,9155
1,CIS,1413,310,776,656,3155
0,APAC,1401,316,803,633,3153
4,All,27520,6084,15599,12530,61733


In [60]:
(raw_test_users
 .fillna('NA')
 .pivot_table(index='group', columns='ab_test', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .rename(columns={'user_id':'ucount'})
)


index,group,interface_eu_test,recommender_system_test,All
0,A,5831,3824,9173
1,B,5736,2877,8269
2,All,11567,6701,16666


In [61]:
(raw_events
 .fillna('NA')
 .pivot_table(index=['event_name','details'], values='user_id', aggfunc='count', margins=True)
 .reset_index()
 .rename(columns={'user_id':'count'})
)


,event_name,details,count
0,login,NA,189552
1,product_cart,NA,62462
2,product_page,NA,125563
3,purchase,4.99,46362
4,purchase,9.99,9530
5,purchase,99.99,5631
6,purchase,499.99,1217
7,All,,440317


In [62]:
raw_marketing


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


Похоже, все на своих местах, кроме `raw_marketing` таблицы. В колонке `regions` у нас записаны значения через запятую, что может вызывать проблемы.

Колонка `details` в таблице `raw_events`, видимо, показывает стоимость покупки. Для событий, которые не соответствуют покупкам, эта колонка будет пустой (ожидаемо).

Наконец, в таблице `raw_tests` наши пользователи из групп А и В принимали участия в двух экспериментах (а некоторые, возможно, участвовали в двух сразу). Нам надо будет это учесть в анализе дальше.

Теперь проверим основные параметры по ТЗ. Начнем с дат в датасетах.

In [63]:
print(
    'User registrations from ', raw_new_users.first_date.min(), ' to ', raw_new_users.first_date.max(), '\n',
    '-' * 50, '\n',
    'User events from ', raw_events.event_dt.min(), ' to ', raw_events.event_dt.max(), '\n',
    '-' * 50, '\n',
    'Marketing events start dates from ', raw_marketing.start_dt.min(), ' to ', raw_marketing.start_dt.max(), '\n',
    'Marketing events end dates from ', raw_marketing.finish_dt.min(), ' to ', raw_marketing.finish_dt.max(),
    sep=''
)


User registrations from 2020-12-07 to 2020-12-23
--------------------------------------------------
User events from 2020-12-07 00:00:33 to 2020-12-30 23:36:33
--------------------------------------------------
Marketing events start dates from 2020-01-25 to 2020-12-30
Marketing events end dates from 2020-02-07 to 2021-01-07


Дата регистрации новых пользователей совпадает с ТЗ: `2020-12-07`, а вот дата окончания не совпадает: `2020-12-23` в датасете против требуемой `2020-12-21`. Надо будет этих пользователей обрезать.

А вот событий у нас не хватает: первое событие происходит `2020-12-07`, а последнее - `2020-12-30`, против требуемой даты `2021-01-04`. По условию, лайфтайм пользователей должен быть 14 дней. Пользователи, которые придут `2020-12-21`, явно не "проживут" достаточно для анализа. Их тоже обрежем, чтобы не искажали картину.

Наконец, у нас было много маркетинговых событий, но только два попадают в интересующие нас даты: New Year Lottery в СНГ и Christmas в Европе и Северной Америке.

Для дальнейших приключений имеет смысл создать суммарный датасет, который мы будем использовать.

In [64]:
# Take users which participated only in one experiment
df_temp = (
    pd.pivot_table(data=raw_test_users, index='user_id', values='ab_test', aggfunc='nunique')
    .reset_index()
    .rename(columns={'ab_test': 'test_count'})
    .loc[lambda df: df.test_count == 1]
)

# From test users select only the ones that had one experiment
df_test_users = raw_test_users[raw_test_users.user_id.isin(df_temp.user_id)].copy()

# Keep the rest of the datasets
df_new_users = raw_new_users.copy().astype({'first_date': 'datetime64[D]'})
df_events = raw_events.copy().astype({'event_dt': 'datetime64[s]'})

# Create a summary dataset
df_summary = (
    raw_events
    .copy()
    .merge(df_new_users, on='user_id', how='left')
    .merge(df_test_users, on='user_id', how='left')
    .assign(
        event_date = lambda df: df.event_dt,
        lifetime = lambda df:
            (pd.to_datetime(df.event_dt) - pd.to_datetime(df.first_date)) / np.timedelta64(1, 'D')
    )
    .loc[lambda df: (df.first_date <= '2020-12-21') & (df.ab_test == 'recommender_system_test')]
    .drop(['first_date', 'ab_test'], axis=1)
    .astype({'event_dt': 'datetime64[s]', 'event_date': 'datetime64[D]'})
)


Посмотрим на количество пользователей в таблице участников эксперимента.

In [65]:
df_temp = (
    df_test_users[df_test_users.ab_test == 'recommender_system_test']
    .pivot_table(index='group', values='user_id', aggfunc='nunique', margins=True)
    .reset_index()
    .rename(columns={'user_id':'user_id_total_ucount'})
    .assign(user_id_pct = lambda df: round(100 * df.user_id_total_ucount / df.user_id_total_ucount.iloc[-1], 1))
)

display(df_temp)


,group,user_id_total_ucount,user_id_pct
0,A,2903,56.9
1,B,2196,43.1
2,All,5099,100.0


Суммарное количество уникальных пользователей в двух группах оказалось меньше 6К. Это связано с изначальной выборкой: в ней у нас было 6.7К человек, но из них 1.6К участвовали в двух экспериментах и нам их пришлось обрезать.

Группы также получились у нас не очень сбалансированные: размах более 10% (около 700 человек разницы).


In [66]:
(df_summary
 .pivot_table(index='group', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename(columns={'user_id':'user_id_active_ucount'})
 .merge(df_temp[['group', 'user_id_total_ucount']], on='group', how='left')
 .assign(user_acitve_pct = lambda df: round(100 * df.user_id_active_ucount / df.user_id_total_ucount, 2))
)


,group,user_id_active_ucount,user_id_total_ucount,user_acitve_pct
0,A,2082,2903,71.72
1,B,706,2196,32.15
2,All,2788,5099,54.68


С активным пользователями тоже не все замечательно, особенно в группе В: там у нас всего 30% людей сделали какое-то действие в рассматриваемый период.

Проверим, как пользователи распределены по регионам.

In [67]:
df_temp = (
    pd.merge(left=df_test_users, right=df_new_users, on='user_id', how='left')
    .loc[lambda df: df.ab_test == 'recommender_system_test']
    .pivot_table(index='region', values='user_id', aggfunc='nunique', margins=True)
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'user_id': 'user_id_ucount'})
    .assign(
        user_id_pct = lambda df: round(100 * df.user_id_ucount / df.user_id_ucount.iloc[-1], 1),
        sort_key = lambda df: df.region == 'All'
    )
    .sort_values(['sort_key', 'user_id_ucount'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

display(df_temp)


index,region,user_id_ucount,user_id_pct
2,EU,4749,93.1
3,N.America,223,4.4
0,APAC,72,1.4
1,CIS,55,1.1
4,All,5099,100.0


Большинство людей в тесте пришли из EU, как мы и ожидали. Посмотрим, какую долю они составляют от общего количества пользователей.

In [69]:
print(
    'Total number of new users from EU: ',
    df_new_users[df_new_users.region == 'EU'].user_id.nunique(), '\n',
    'Number of new users from EU in the test: ', 
    df_temp[df_temp.region == 'EU'].user_id_ucount.sum(), '\n',
    'Percentage of new users from EU in the test: ',
    round(100 * df_temp[df_temp.region == 'EU'].user_id_ucount.sum()
    / df_new_users[df_new_users.region == 'EU'].user_id.nunique(), 2),
    sep=''
)


Total number of new users from EU: 46270
Number of new users from EU in the test: 4749
Percentage of new users from EU in the test: 10.26


Пользователи из EU в тесте составляют 10%. Это снова не то, что мы бы хотели, однако, здесь причина та же, что и в предыдущей секции - мы убрали пользователей, которые участвовали в двух тестах одновременно.

## Исследовательский анализ данных

Посмотрим, какие основные зависимости в данных мы можем найти: сколько у нас пользователей, откуда они приходят, как себя ведут в игре и прочее.

### Анализ количества пользователей